# Trading Dashboard (app)
Human-Model trading on trade subgroups - H2

In [1]:
import pandas as pd 
import numpy as np 
import sqlite3 as sql
import pickle

# Plotly
import plotly.graph_objects as go

# Web app
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
import dash_table

database_name = "FX_data/database.db"

In [2]:

conn = sql.connect('FX_data/database.db')
cur = conn.cursor()
df = pd.read_sql_query("SELECT * FROM eda_data_LR ",conn)
df['Date'] = pd.to_datetime(df['Date'])
df.set_index(['Date'],inplace=True)
conn.close()

#df = pd.read_sql_query("SELECT * FROM profitable_subgroups", conn)
df.head(1)

,Target,Prediction,Correct,% Buy,% Sell,Price,Open,High,Low,Close,Prob Buy,Prob Sell,Balance,Profit,comp_0,comp_1,t-SNE_1,t-SNE_2,Cluster,Confidence %
Date,,,,,,,,,,,,,,,,,,,,
2004-03-01,0,1,0,37.25,62.75,0.8034,0.755824,0.762779,0.758515,0.76933,0.364423,0.635577,9903.846154,-96.153846,-0.019401,0.017532,22.393435,-12.548299,5,62.75


## V1.
- [Options Table](https://github.com/plotly/dash-sample-apps/blob/master/apps/dash-tsne/demo.py)
- [html colours](https://www.bitdegree.org/learn/html-color)
- [_interface app example_](https://github.com/plotly/dash-sample-apps/blob/master/apps/dash-tsne/demo.py#L357)
- [template multi plot](https://community.plot.ly/t/how-to-manage-the-layout-of-division-figures-in-dash/6484/3)
- [Price chart example](https://plot.ly/python/time-series/)
    - [Candle sticks](https://plot.ly/python/candlestick-charts/)


In [71]:
# pd.DatetimeIndex(df.reset_index()['Date']).quarter.to_list()
# list(df["Cluster"].unique())


# For time-series
#df['Price'].loc[(df.index >= '2004-03-02') & (df.index <= '2005-03-05')].plot()

#df[df["Prediction"].isin([1])]

#year_options



#[{'label': i, 'value': i} for i in list(df["Cluster"].unique())]

[{'label': i, 'value': i} for i in list(['eda_data_LR','protitable_subgroups'])]
dataset_options[1]['label']   #['protitable_subgroups']

action_options

# [{'label':'All Trades','value':'eda_data_LR'}, {'label':'Profitable Subgroups','value':'profitable_subgroups'}]
dataset_options

[{'label': 'eda_data_LR', 'value': 'eda_data_LR'},
 {'label': 'profitable_subgroups', 'value': 'profitable_subgroups'}]

### NOT WORKING - LAYOUT
NoLayoutException: Layout must be a dash component or a function that returns a dash component.


In [62]:
cluster_options = [{'label': i, 'value': i} for i in list(df["Cluster"].unique())]
year_options = pd.DatetimeIndex(df.reset_index()['Date']).year #.year
action_options = [{'label': i, 'value': i} for i in list(df["Prediction"].unique())]
dataset_options = [{'label':'All Trades','value':'eda_data_LR'}, {'label':'Profitable Subgroups','value':'profitable_subgroups'}]
url_path ="dash-logo.png" #"http://www.digicom.com.cy/wp-content/uploads/2013/11/Digicom.png"

# Methods for creating components in the layout code
def Card(children, **kwargs):
    return html.Section(children, className="card-style")


# DEFINE HTML TEMPLATE
app = dash.Dash() # Boostrap CSS.
app.css.append_css({'external_url': 'https://codepen.io/amyoshino/pen/jzXypZ.css'}) 

# app = dash.Dash(
#     __name__, meta_tags=[{"name": "viewport", "content": "width=device-width"}]
# )
# server = app.server


app.layout = html.Div(
    #[
        
    # ===========================     NAME      =============================== #
    #html.Div(
        className="row",
        style={"max-width": "100%", "font-size": "1.5rem", "padding": "0px 0px"}, #,\
               #"background-color": "rgb(134, 145, 191)"}, #"#f9f9f9"
        children=[
            # WRITE HEADER 
            html.Div( 
                className= "row header",  #"four columns",
                id = "app_header",
                style = {"background-color": "rgb(134, 145, 191)"},
                children=[ 
                    # IMAGE -APP HEADER
                    html.Div([
                        html.Img(
                            src=app.get_asset_url(url_path),
                            className="logo",
                            id="plotly-image",
                        ),
                    ], className="three columns header_img"),
                    # TITLE -HEADER    
                    html.Div([
                        html.H1(
                            " USD/EUR TRADES ", 
                            style={'textAlign': 'center'},                        
                            className="header_title",
                            id="app-title"
                        ),
                    ], className="nine columns header_title"),
                ],), # end children
#             # WRITE DESCRIPTION
#             html.Div(
#                 className="row background",
#                 id="demo-explanation",
#                 style={"padding": "50px 45px"},
#                 children=[
#                     html.Div(
#                         id="description-text", 
#                         children=dcc.Markdown(demo_intro_md)
#                     ),
#                     # ADDITIONAL INFOR DISPLAYED UPON SELECTION
#                     html.Div(
#                         html.Button(id="learn-more-button", children=["Learn More"])
#                     ),
#                 ],
#             ), 
            
            # SECTION 1
            html.Div(
            className="row background",
            style={"padding": "10px"},
                children=[
                    html.Div(
                        className="four columns",
                        children=[ 
                                # USER OPTIONS TABLE
                            html.Section(
                                children=[                
                                # Dataset ( All trades / Profitable Subgroup)
                                    dcc.Dropdown(id='dataset',
                                                 options= dataset_options,
                                                 value= 'eda_data_LR',
                                                 style={"display": "block","margin-left": "auto","margin-right": "auto","width": "40%"},
                                                 placeholder= "Select a dataset", 
                                                 searchable=False,
                                                 clearable=False,
                                                ),  


                                    # Cluster Dropdown 
                                    dcc.Dropdown(id='cluster_1',
                                                 options=cluster_options, 
                                                 multi=True,
                                                 value= list(range(0,10)), #list(df["Cluster"].unique().sort_values()),
                                                 style={"display": "block","margin-left": "auto","margin-right": "auto","width": "40%"}
                                                 ),                
                                    # Trade Actions - Buy/Sell
                                    dcc.Checklist(id='action_1',
                                                  options=action_options,
                                                  value= [0,1] ,             ),
                                    # DateRange - Years
                                    html.Div(
                                        id='slider_selection',
                                        children=[
                                            dcc.RangeSlider( id = "date_1",
                                                             updatemode = "drag", 
                                                             min = year_options.min(),
                                                             max = year_options.max(),
                                                             marks = {i: '{}'.format(i) for i in year_options.unique().tolist()},
                                                             value= [2005,2006],         
                                            ),
                                        ],
                                    ),
                                ],className="card-style",
                            ), # END USER OPTIONS TABLE
                        ],
                    ),

                    html.Div( 
                        className="eight columns",
                        children=[      
                                # 3d ScatterPlot
                            dcc.Graph(
                                id = "scatter_trader", 
                                style={"height": "98vh"},
                            ),
                        ],
                    ),
                ], #className='row background',
            ),   #className='twelver columns'),
        ],
    ),
#], className="container")    

#2 DateRange - Years
# html.Div(id='slider_selection'),
# =========================== PLOT 2 ======================================== #
# """
# html.Div([
# dash_table.DataTable(id = "table_1",
#                      columns = [{"name": i, "id": i} for i in df.iloc[:, 4:25]], #4-25
#                      editable=True,  
# #                                      n_fixed_columns=2,
# #                                      style_table={'maxWidth': '1500px'},
#                      row_selectable="multi",       
#                      data  = df.iloc[:, 4:6].to_json(),
#                      virtualization = True,

# #                                      virtualization = True,
# #                                      page_action = "none",
#                     ),

#         ],className='twelver columns'),
# #className='four columns'), # Data table 4/12 columns in Tab1
# """

#])     
#], className="container")


"===================================================================================================="
"===================================================================================================="

# User selection --> What is returned 
@app.callback([Output('scatter_trader', 'figure'),
               Output('slider_selection', 'children'), ],
               #Output('table_1', 'data'),  ], # Dynamic data table 
    
              [Input ('dataset','value'),
               Input('cluster_1','value'),
               Input('action_1','value'), 
               Input('date_1','value'),           ]   ) 

def scatter_3D (datasetD, clusterD, actionD, rangeD ):  # just my names (According to order in callback)
    
    # Call data from SQL db
    conn = sql.connect(database_name)
    cur = conn.cursor()
    df = pd.read_sql_query("SELECT * FROM {} ".format(datasetD), conn)  #FROM eda_data_LR
    df["Date"] = pd.DatetimeIndex(df["Date"]).year
    conn.close()    
    
    dataset = df[(df["Date"] >= rangeD[0]) &  # Slice data according to lower year range 
                   (df["Date"] <= rangeD[1]) & # Slice data according to upper year range
                   (df["Cluster"].isin(clusterD)) & # Slice data according to quarter                       
                   (df["Prediction"].isin(actionD))  # Slice data according to sector                            
                  ]  

    trace1 = go.Scatter3d(x = dataset["t-SNE_1"],   # Exponential PCA
                          y = dataset["t-SNE_2"],
                          z = dataset["Profit"], # np.exp(dataset["y_return"])*10,  #)
                          mode = "markers+text",
                          hovertext =  round(dataset[["Profit","Confidence %"]],2).values.tolist(), #dataset[["Profit"]].values.tolist(),  #,"Confidence %"
                          hoverinfo= "text",
                          marker = dict(size= df["Confidence %"]/4, 
                                        sizemin = 1.5,  # min size
                                        color= dataset["Cluster"],  # THIS CAN BE DIFFERENT
                                        colorscale = "Portland",
                                        colorbar = {"thickness": 10, "len": 0.5, "x": 0.8, "y": 0.6, },
                                        symbol = list(np.where(dataset['Correct'] == 1, 'diamond', 'cross')), #square #-open
#                                         symbol = ['cross', 'diamond', 'square', 'square-open', # number markers == sectors
#                                                   'square', 'diamond-open', 'cross', 'x', 'square','square', 'circle-open'],
                                        showscale = False,
                                       ),  
                         )

    data = [ trace1 ]    
    layout = go.Layout( 
         title = "Analyze Trader Subgroups",
             # UX
             height = 1000,  #1500,
             width = 1000,  #1500,
             scene = dict( # UI
                          aspectmode = "cube",  
                          hovermode = "closest",
                          dragmode = "turntable", #  "orbit" | "turntable" | "zoom" | "pan" | False
                          xaxis = {"title": "t-SNE_1", },
                          yaxis = {"title": "t-SNE_2", },
                          zaxis = {"title": "Profit", },  )   )   

    fig =( {"data":data,
            "layout":layout})
    
    return fig , "Date Range {} - {}.".format(str(rangeD[0]),str(rangeD[1])) #, data_table

if __name__ == '__main__':
    app.run_server()

NoLayoutException: Layout must be a dash component or a function that returns a dash component.

### V2 - works but layout fucked

## V3
- [Cards](https://github.com/plotly/dash-sample-apps/blob/854b8df105e24b04f6b3f9124c016f1e01b23691/apps/dash-clinical-analytics/app.py#L67)
- [__Cards Doc__](https://dash-bootstrap-components.opensource.faculty.ai/l/components/card)
- 

## V4 - Works
- top user options section is not on the right of scatter plot
- the plot is below 

### V5 - Works 
- side by side 
- need to beautify left side card

In [13]:
#import dash_bootstrap_components as dbc


cluster_options = [{'label': i, 'value': i} for i in list(df["Cluster"].unique())]
year_options = pd.DatetimeIndex(df.reset_index()['Date']).year #.year
action_options = [{'label': k, 'value': v} for k,v in {('Sell Signal',0),('Buy Signal',1)}]#[{'label': i, 'value': i} for i in list(df["Prediction"].unique())]
forecast_options = [{'label': k, 'value': v} for k,v in {('Forecasted Wrongly',0),('Forecasted Correctly',1)}]#[{'label': i, 'value': i} for i in list(df["Correct"].unique())]
dataset_options = [{'label':'All Trades','value':'eda_data_LR'}, {'label':'Profitable Subgroups','value':'profitable_subgroups'}]

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']  #FIX 
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)


# DEFINE HTML TEMPLATE
#app.css.append_css({'external_url': 'https://codepen.io/amyoshino/pen/jzXypZ.css'}) 

#app = dash.Dash() # Boostrap CSS.

app.layout = \
html.Div(
    [
        dcc.Store(
            id="aggregate_data"
        ),
        # TITLE BANNER
        html.Div(
            [
            
            html.Div(
                [
                    html.H2(
                    "TRADES APP"
                    ),
                    html.H4(
                    "TEST OVERVIEW"
                    ),
                ],
                className="eight columns",
            ),
            html.Img(
                src="https://s3-us-west-1.amazonaws.com/plotly-tutorials/logo/new-branding/dash-logo-by-plotly-stripe.png",
                style={"height": "10%",
                       "width": "10%"},
                className="four columns",                   
            ),
            ],
            id="header",
            className="row",
        ),       
        # USER OPTION PANNEL SEPARATE
        html.Div(       
            [
                html.Div(
                    [

#                     html.Div(
#                         [
                    # CARD 2
                        #1 Dataset ( All trades / Profitable Subgroup)
                        html.P(
                            "Select All Clusters / Profitable Clusters:",
                            className="control_label",
                        ),
                        dcc.Dropdown(
                            id='dataset',
                            options= dataset_options,
                            value= 'eda_data_LR',
                            #style={"display": "block","margin-left": "0","width": "40%"},
                            placeholder= "Select a dataset", 
                            searchable=False,
                            clearable=False,
                            className="dcc_control",
                        ),  
                        #2 Cluster Dropdown 
                        html.P(
                            "Cluster Dropdown:",
                            className="control_panel",
                        ), 
                        dcc.Dropdown(
                            id='cluster_1',
                            options=cluster_options, 
                            multi=True,
                            value= list(range(0,10)), #list(df["Cluster"].unique().sort_values()),
                            #style={"display": "block","margin-left": "0","width": "70%"},
                            className="dcc_control",                      
                        ),                
                        #3 Trade Actions - Buy/Sell
                        html.P(
                            "Select Trade Type:",
                            className="control_panel",
                        ), 
                        dcc.Checklist(
                            id='action_1',
                            options=action_options,
                            value= [0,1],                            
                            #style={"display": "block","margin-left": "0","width": "30%"},     
                            className="dcc_control",  
                        ),

                        # Forecasts - Correct/Wrong
                        html.P(
                            "Filter Correct/False Forecasts:",
                            className="control_panel",
                        ), 
                        dcc.Checklist(
                            id='forecast_1',
                            options=forecast_options,
                            value= [0,1],                            
                            #style={"display": "block","margin-left": "0","width": "30%"},     
                            className="dcc_control",  
                        ),                        

                        #4 Data range slider - Years
                        html.P("Select Year:",
                            className="control_panel",
                        ),                     
                        dcc.RangeSlider( 
                            id = "date_1",
                            updatemode = "drag", 
                            min = year_options.min(),
                            max = year_options.max(),
                            marks = {i: '{}'.format(i) for i in year_options.unique().tolist()},
                            value= [2005,2006],        
                            className="dcc_control", 
                        ), 
                        html.Div(
                            id='slider_selection',
#                             style={"display": "block","margin-left": "auto","width": "30%"},
                        ), 
                    ],
                    #id='slider_selection',
                    #style={'width': '30%', 'display': 'inline-block'},
                    className="four columns",
                ),
                # SCATTERPLOT SEPARATE
                html.Div(
                    [    
                        html.Div(
                            [
                                html.B("Scatterplot: Trades"),
                                html.Hr(),
                                dcc.Graph(
                                    id = "scatter_trader",  
                                    style={"display": "block","margin-right": "0","width": "70%"},
                                        ),
                            ],
                            id = "countGraphContainer",
                            className="pretty_container",
                        ),
                    ],
                    #id="rightCol",
                    #style={'width': '70%', 'display': 'inline-block'},
                    className="eight columns",
                ),    
            ],
            className="row",

        ),
    ],
)    






In [14]:
# User selection --> What is returned 
@app.callback([Output('scatter_trader', 'figure'),
               Output('slider_selection', 'children'), ],
               #Output('table_1', 'data'),  ], # Dynamic data table 
    
              [Input ('dataset','value'),
               Input('cluster_1','value'),
               Input('action_1','value'), 
               Input('forecast_1','value'),
               Input('date_1','value'),           ]   ) 

def scatter_3D (datasetD, clusterD, actionD, forecastD, rangeD ):  # just my names (According to order in callback)
    
    # Call data from SQL db
    conn = sql.connect(database_name)
    cur = conn.cursor()
    df = pd.read_sql_query("SELECT * FROM {} ".format(datasetD), conn)  #FROM eda_data_LR
    df["Date"] = pd.DatetimeIndex(df["Date"]).year
    conn.close()    
    
    dataset = df[(df["Date"] >= rangeD[0]) &  # Slice data according to lower year range 
                   (df["Date"] <= rangeD[1]) & # Slice data according to upper year range
                   (df["Cluster"].isin(clusterD)) & # Slice data according to quarter                       
                   (df["Prediction"].isin(actionD)) &
                   (df["Correct"].isin(forecastD))# Slice data according to sector    
                  ]  

    trace1 = go.Scatter3d(x = dataset["t-SNE_1"],   # Exponential PCA
                          y = dataset["t-SNE_2"],
                          z = dataset["Profit"], # np.exp(dataset["y_return"])*10,  #)
                          mode = "markers+text",
                          hovertext =  round(dataset[["Profit","Confidence %"]],2).values.tolist(), #dataset[["Profit"]].values.tolist(),  #,"Confidence %"
                          hoverinfo= "text",
                          marker = dict(size= df["Confidence %"]/4, 
                                        sizemin = 1.5,  # min size
                                        color= dataset["Cluster"],  # THIS CAN BE DIFFERENT
                                        colorscale = "Portland",
                                        colorbar = {"thickness": 10, "len": 0.5, "x": 0.8, "y": 0.6, },
                                        symbol = list(np.where(dataset['Correct'] == 1, 'diamond', 'cross')), #square #-open
                                        showscale = False,
                                       ),  
                         )
    data = [ trace1 ]    
    layout = go.Layout( 
         title = "Analyze Trader Subgroups",
             # UX
             height = 1000,  #1500,
             width = 1000,  #1500,
             scene = dict( # UI
                          aspectmode = "cube",  
                          hovermode = "closest",
                          dragmode = "turntable", #  "orbit" | "turntable" | "zoom" | "pan" | False
                          xaxis = {"title": "t-SNE_1", },
                          yaxis = {"title": "t-SNE_2", },
                          zaxis = {"title": "Profit", },  )   )   

    fig =( {"data":data,
            "layout":layout})
    
    return fig , "Date Range {} - {}.".format(str(rangeD[0]),str(rangeD[1])) #, data_table

if __name__ == '__main__':
    app.run_server()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [13/Feb/2020 19:00:29] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [13/Feb/2020 19:00:29] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [13/Feb/2020 19:00:30] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [13/Feb/2020 19:00:30] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [13/Feb/2020 19:00:35] "GET /_favicon.ico?v=1.7.0 HTTP/1.1" 200 -


## Backup

In [ ]:
- [ ] App 7 More User customization %  
  - [ ] [Insert data table](https://dash.plot.ly/datatable), [Dynamic table](https://towardsdatascience.com/how-to-build-a-complex-reporting-dashboard-using-dash-and-plotl-4f4257c18a7f). [Dynamic Table2](https://dash-gallery.plotly.host/dash-medical-provider-charges/). [Dynamic Table3](https://dash-gallery.plotly.host/dash-drug-discovery/).
      - [ ] Use regex on _ to get feature column names from df
      - [ ] [Visrtualization improves frame rate](https://dash.plot.ly/datatable/virtualization).
  - [ ] [Soft Clustering](https://plot.ly/python/v3/3d-point-clustering/)
  - [ ] [Animate time-series](https://plot.ly/python/animations/#animated-figures-with-graph-objects)
  - [ ] [Cross Filtering](https://dash.plot.ly/interactive-graphing)
      - [ ] Tabs for different data: [Fundamental, Economic, Technical]
      - [ ] Hover over table to get PCA explained variance by feature